In [1]:
import digits
import tensorflow as tf
from time import time
import itertools as it
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
DATA_PATH = 'digits.csv'

LEARNING_RATE = 0.05
BATCH_SIZE = 32
EPOCHS = 30
MODEL_DIR = 'cnn/model_{}'

In [3]:
labels, pixels = digits.read(DATA_PATH)
pixels_sq = pixels.reshape(-1, 28, 28)

In [4]:
wew = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
train_label, test_label, train_pixel, test_pixel = train_test_split(labels, pixels_sq, test_size=0.1, 
                                                                    stratify=labels, shuffle=True, random_state=0)
train_label, dev_label, train_pixel, dev_pixel = train_test_split(train_label, train_pixel, test_size=0.11111111, 
                                                                  stratify=train_label, shuffle=True, random_state=0)

In [6]:
train_label.shape, dev_label.shape, test_label.shape

((8000,), (1000,), (1000,))

In [7]:
train_pixel.shape, dev_pixel.shape, test_pixel.shape

((8000, 28, 28), (1000, 28, 28), (1000, 28, 28))

In [8]:
num_labels = len(set(labels))
num_labels

10

In [9]:
def model_fn(features, labels, mode):
    c1 = tf.layers.conv2d(features, 4, 3, activation=tf.nn.relu) # 26 x 26
    c2 = tf.layers.conv2d(c1, 4, 3, activation=tf.nn.relu) # 24 x 24
    m1 = tf.layers.max_pooling2d(c2, 2, 2) # 12 x 12
    c3 = tf.layers.conv2d(m1, 8, 3, activation=tf.nn.relu) # 10 x 10
    c4 = tf.layers.conv2d(c3, 8, 3, activation=tf.nn.relu) # 8 x 8
    c5 = tf.layers.conv2d(c4, 8, 3, activation=tf.nn.relu) # 6 x 6
    m2 = tf.layers.max_pooling2d(c5, 2, 2) # 3 x 3
    fc = tf.layers.flatten(m2)
    logits = tf.layers.dense(fc, num_labels)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        proba = tf.nn.softmax(logits, axis=-1)
        predictions = {
            'logits': logits,
            'probabilities': proba,
            'labels': tf.argmax(proba, axis=-1),
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.softmax_cross_entropy(labels, logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss)
    
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [10]:
def td_input_fn(pixels, labels, batch_size, epochs, is_training, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices((pixels, labels))
    
    def preprocessing(p, lbl):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        lbl_1h = tf.one_hot(lbl, num_labels, dtype=tf.float32)
        return p3d, lbl_1h
    
    ds = ds.map(preprocessing, num_parallel_calls=8)
    
    if is_training:
        ds = ds.shuffle(buffer * batch_size)
        
    ds = ds.batch(batch_size)
    ds = ds.repeat(epochs)
    ds = ds.prefetch(buffer)
    
    return ds

In [11]:
def pred_input_fn(pixels, batch_size, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices(pixels)
    
    def preprocessing(p):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        return p3d
        
    ds = ds.map(preprocessing, num_parallel_calls=8)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer)
    
    return ds

In [12]:
model_dir = MODEL_DIR.format(int(time()))
config = tf.estimator.RunConfig(tf_random_seed=0,
                                save_summary_steps=50,
                                save_checkpoints_steps=250,
                                keep_checkpoint_max=None)

clf = tf.estimator.Estimator(model_fn, model_dir, config)

INFO:tensorflow:Using config: {'_model_dir': 'cnn/model_1539170697', '_tf_random_seed': 0, '_save_summary_steps': 50, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A782E20AC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
train_input_fn = lambda: td_input_fn(train_pixel, train_label, BATCH_SIZE, EPOCHS, True)
dev_input_fn = lambda: td_input_fn(dev_pixel, dev_label, BATCH_SIZE, 1, False)

train_spec = tf.estimator.TrainSpec(train_input_fn)
eval_spec = tf.estimator.EvalSpec(dev_input_fn, None, start_delay_secs=0.1, throttle_secs=0.1)

In [14]:
tf.estimator.train_and_evaluate(clf, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 250 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cnn/model_1539170697\model.ckpt.
INFO:tensorflow:cnn/model_1539170697\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:loss = 2.245653, step = 0
INFO:tensorflow:global_step/sec: 151.681
INFO:tensorflow:loss = 1.1502795, step = 100 (0.659 sec)
INFO:tensorflow:global_step/sec: 231.587
INFO:tensorflow:loss = 0.24385418, step = 200 (0.433 sec)
INFO:tensorflow:Saving checkpoints for 250 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-10-11:25:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn/model_1539170697\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-10-11:25:24
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.16451737
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: cnn/model_1539170697\model.ckpt-2000
INFO:tensorflow:global_step/sec: 77.3983
INFO:tensorflow:loss = 0.005487552, step = 2000 (1.292 sec)
INFO:tensorflow:global_step/sec: 257.757
INFO:tensorflow:loss = 0.04278197, step = 2100 (0.389 sec)
INFO:tensorflow:global_step/sec: 250.667
INFO:tensorflow:loss = 0.18980974, step = 2200 (0.399 sec)
INFO:tensorflow:Saving checkpoints for 2250 into cnn/model_1539170697\model.ckpt.
INFO:tensorflow:cnn/model_15391

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-10-11:25:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn/model_1539170697\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-10-11:25:39
INFO:tensorflow:Saving dict for global step 4000: global_step = 4000, loss = 0.16567063
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: cnn/model_1539170697\model.ckpt-4000
INFO:tensorflow:global_step/sec: 73.154
INFO:tensorflow:loss = 0.03664662, step = 4000 (1.366 sec)
INFO:tensorflow:global_step/sec: 268.118
INFO:tensorflow:loss = 0.044203468, step = 4100 (0.373 sec)
INFO:tensorflow:global_step/sec: 237.039
INFO:tensorflow:loss = 0.24009037, step = 4200 (0.421 sec)
INFO:tensorflow:Saving checkpoints for 4250 into cnn/model_1539170697\model.ckpt.
INFO:tensorflow:cnn/model_153917

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-10-11:25:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn/model_1539170697\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-10-11:25:55
INFO:tensorflow:Saving dict for global step 6000: global_step = 6000, loss = 0.16563328
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6000: cnn/model_1539170697\model.ckpt-6000
INFO:tensorflow:global_step/sec: 75.5017
INFO:tensorflow:loss = 0.110153265, step = 6000 (1.325 sec)
INFO:tensorflow:global_step/sec: 267.018
INFO:tensorflow:loss = 0.077894375, step = 6100 (0.374 sec)
INFO:tensorflow:global_step/sec: 245.125
INFO:tensorflow:loss = 0.00601664, step = 6200 (0.408 sec)
INFO:tensorflow:Saving checkpoints for 6250 into cnn/model_1539170697\model.ckpt.
INFO:tensorflow:cnn/model_1539

({'loss': 0.18457806, 'global_step': 7500}, [])

In [15]:
test_input_fn = lambda: pred_input_fn(test_pixel, BATCH_SIZE)

test_pred = clf.predict(test_input_fn, predict_keys=['labels'])
test_pred = [p['labels'] for p in test_pred]
test_pred = np.asarray(test_pred)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn/model_1539170697\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [16]:
print(classification_report(test_label, test_pred))

             precision    recall  f1-score   support

          0       0.99      0.97      0.98        99
          1       0.99      0.98      0.99       110
          2       0.95      0.99      0.97       104
          3       0.96      1.00      0.98       101
          4       0.96      0.96      0.96        97
          5       0.95      0.98      0.96        91
          6       0.99      0.99      0.99       100
          7       0.99      0.95      0.97       104
          8       0.99      0.92      0.95        95
          9       0.94      0.97      0.96        99

avg / total       0.97      0.97      0.97      1000



In [17]:
cm = confusion_matrix(test_label, test_pred)
cm

array([[ 96,   0,   1,   1,   0,   1,   0,   0,   0,   0],
       [  0, 108,   0,   0,   2,   0,   0,   0,   0,   0],
       [  0,   0, 103,   0,   1,   0,   0,   0,   0,   0],
       [  0,   0,   0, 101,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  93,   0,   0,   1,   0,   3],
       [  0,   0,   0,   1,   0,  89,   0,   0,   0,   1],
       [  0,   0,   0,   0,   0,   0,  99,   0,   1,   0],
       [  0,   1,   2,   0,   0,   0,   0,  99,   0,   2],
       [  1,   0,   2,   1,   1,   2,   1,   0,  87,   0],
       [  0,   0,   0,   1,   0,   2,   0,   0,   0,  96]], dtype=int64)